In [7]:
import pandas as pd
import math
import numpy as np
import random
import sqlite3
import time
from os.path import expanduser
import os

In [8]:
CONVERTED_DATABASE_NAME = '/media/big-ssd/experiments/P3856/converted-databases/exp-P3856-run-P3856_YHE211_1_Slot1-1_1_5104-converted.sqlite'
# CONVERTED_DATABASE_NAME = '/Users/darylwilding-mcbride/Downloads/experiments/P3856/converted-databases/exp-P3856-run-P3856_YHE211_1_Slot1-1_1_5104-converted.sqlite'

In [9]:
FRAME_TYPE_MS1 = 0
rt_lower = 1650
rt_upper = 2200
scan_limit = 400
segment_mz_lower = 780
segment_mz_upper = 800

In [10]:
start_run = time.time()

db_conn = sqlite3.connect(CONVERTED_DATABASE_NAME)
raw_df = pd.read_sql_query("select frame_id,mz,scan,intensity,retention_time_secs from frames where frame_type == {} and retention_time_secs >= {} and retention_time_secs <= {} and scan >= {} and mz >= {} and mz <= {}".format(FRAME_TYPE_MS1, rt_lower, rt_upper, scan_limit, segment_mz_lower, segment_mz_upper), db_conn)
db_conn.close()

stop_run = time.time()
print("loaded {} points in {} seconds".format(len(raw_df), round(stop_run-start_run,1)))

loaded 6763150 points in 45.5 seconds


In [12]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6763150 entries, 0 to 6763149
Data columns (total 5 columns):
 #   Column               Dtype  
---  ------               -----  
 0   frame_id             int64  
 1   mz                   float64
 2   scan                 int64  
 3   intensity            int64  
 4   retention_time_secs  float64
dtypes: float64(2), int64(3)
memory usage: 258.0 MB


In [13]:
raw_df.memory_usage(deep=True)

Index                       128
frame_id               54105200
mz                     54105200
scan                   54105200
intensity              54105200
retention_time_secs    54105200
dtype: int64

In [14]:
int_columns = ['frame_id','scan','intensity']
raw_df[int_columns] = raw_df[int_columns].apply(pd.to_numeric, downcast="unsigned")

In [15]:
float_columns = ['mz','retention_time_secs']
raw_df[float_columns] = raw_df[float_columns].apply(pd.to_numeric, downcast="float")

In [16]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6763150 entries, 0 to 6763149
Data columns (total 5 columns):
 #   Column               Dtype  
---  ------               -----  
 0   frame_id             uint16 
 1   mz                   float32
 2   scan                 uint16 
 3   intensity            uint16 
 4   retention_time_secs  float32
dtypes: float32(2), uint16(3)
memory usage: 90.3 MB


#### Pickle format

In [17]:
raw_df.to_pickle('{}/raw_df.pkl'.format(expanduser('~')))

In [18]:
start_run = time.time()
raw_df = pd.read_pickle('{}/raw_df.pkl'.format(expanduser('~')))
stop_run = time.time()
print("loaded {} points in {} seconds".format(len(raw_df), round(stop_run-start_run,1)))

loaded 6763150 points in 0.1 seconds


#### HDF format

In [6]:
HDF_DATABASE_NAME = '/media/big-ssd/experiments/P3856/converted-databases/exp-P3856-run-P3856_YHE211_1_Slot1-1_1_5104-converted.hdf'
# HDF_DATABASE_NAME = '{}/Downloads/exp-P3856-run-P3856_YHE211_1_Slot1-1_1_5104-converted.hdf'.format(expanduser('~'))
HDF_DATABASE_NAME

'/media/big-ssd/experiments/P3856/converted-databases/exp-P3856-run-P3856_YHE211_1_Slot1-1_1_5104-converted.hdf'

In [ ]:
start_run = time.time()

raw_df = pd.read_hdf(HDF_DATABASE_NAME, key='frames', columns=['frame_id','mz','retention_time_secs','scan','intensity'], where=["frame_type == {} and retention_time_secs >= {} and retention_time_secs <= {} and scan >= {} and mz >= {} and mz <= {}".format(FRAME_TYPE_MS1, rt_lower, rt_upper, scan_limit, segment_mz_lower, segment_mz_upper)])

stop_run = time.time()
print("loaded {} points in {} seconds".format(len(raw_df), round(stop_run-start_run,1)))

In [6]:
raw_df.sample(n=5)

,frame_id,mz,retention_time_secs,scan,intensity
18322346,14867,927.930839,1677.435519,425,96
16932351,14837,668.816649,1674.258781,668,65
8886646,14677,812.904921,1657.325852,501,41
13571428,15272,892.465779,1720.304310,345,106
16522334,15332,1146.831015,1726.658903,432,69


In [7]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46945010 entries, 5563291 to 2672451
Data columns (total 5 columns):
 #   Column               Dtype  
---  ------               -----  
 0   frame_id             int64  
 1   mz                   float64
 2   retention_time_secs  float64
 3   scan                 int64  
 4   intensity            int64  
dtypes: float64(2), int64(3)
memory usage: 2.1 GB


In [8]:
start_run = time.time()
for i in range(200):
    subset_df = raw_df[(raw_df.mz >= 700) & (raw_df.mz <= 720) & (raw_df.scan >= 500) & (raw_df.scan <= 550) & (raw_df.retention_time_secs >= 1700) & (raw_df.retention_time_secs <= 1760)]
stop_run = time.time()
print("total running time: {} seconds".format(round(stop_run-start_run,1)))

total running time: 78.8 seconds


In [13]:
start_run = time.time()
for i in range(10000):
    rt_df = subset_df.groupby(['frame_id'], as_index=False).intensity.sum()
stop_run = time.time()
print("total running time: {} seconds".format(round(stop_run-start_run,1)))

total running time: 16.0 seconds


#### comparing with pickle